In [565]:
import xml.etree.ElementTree as ET
import os
import dill
import pandas as pd
from datetime import datetime
import operator
import re

In [566]:
os.chdir('/Users/longy/Documents/TDI/capstone')

In [567]:
# 2008-08-04T22:52:00.650
xml = ET.iterparse('Posts.xml', events = ('start', 'end'))
# PostTypeId, CreationDate, 
posts = []
for event, elem in iter(xml):
    if len(posts) < 500000:  
        tag = elem.tag
        value = elem.attrib
        if event == 'start':
            if tag=='row':  
                d = dict()
                if 'OwnerUserId' in value:
                    d['OwnerUserId'] = value['OwnerUserId']
                elif 'OwnerDisplayName' in value:
                    d['OwnerUserId'] = value['OwnerDisplayName']
                else:
                    d['OwnerUserId'] = None
                if value['PostTypeId']=='1':
                    d['Tags'] = value['Tags']
                    d['ParentId'] = None
                    d['Id'] = value['Id']
                    d['CreationDate'] = datetime.strptime(value['CreationDate'], '%Y-%m-%dT%H:%M:%S.%f')
                    d['Body'] = value['Body']
                    d['Title'] = value['Title']
                    d['Score'] = value['Score']
                    d['PostTypeId'] = value['PostTypeId']
                elif value['PostTypeId']=='2':
#                     d['Tags'] = value['Tags']
                    d['ParentId'] = value['ParentId']
                    d['Id'] = value['Id']
                    d['CreationDate'] = datetime.strptime(value['CreationDate'], '%Y-%m-%dT%H:%M:%S.%f')
                    d['Body'] = value['Body']
                    d['Title'] = None
                    d['Score'] = value['Score']
                    d['PostTypeId'] = value['PostTypeId']
                posts.append(d)
        elem.clear()
    else:
        break

dill.dump(posts, open('posts.pkd', 'w'))
posts = dill.load(open('posts.pkd', 'r'))
# root = doc.getroot()
# <row Id="" PostTypeId="" CreationDate="" Score="" ViewCount="" Body="" OwnerUserId="5" LastEditorUserId="" LastEditDate="" LastActivityDate="" Title="" Tags="" AnswerCount="" CommentCount="" />

In [568]:
names = list(posts[0].keys())
df = pd.DataFrame(posts, columns = names)

In [589]:
df.describe()

,Body,Tags,Title,ParentId,Score,PostTypeId,OwnerUserId,CreationDate,Id
count,500000,111439,111439,388561,500000,500000,499080,500000,500000
unique,499887,77693,111378,109111,1475,2,40553,499769,500000
top,<p>Anyone have any idea? And any open source s...,<c#>,Need help configuring IIS to serve 2.0 web ser...,184618,0,2,22656,2009-03-11 13:25:05.237000,527446
freq,4,572,4,300,105508,388561,2624,2,1
first,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008-07-31 21:42:52.667000,NaN
last,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009-03-29 20:16:23.710000,NaN


In [569]:
byOwnerID = df.groupby('OwnerUserId')
Owner342 = byOwnerID.get_group('342')
#Questions raised by owner342
Owner342.loc[(Owner342.PostTypeId == '1'), 'Id']

1437        3448
2490        5892
3280        7614
8241       17117
14081      28302
26416      48293
26910      49196
28498      51548
31402      56149
33419      59280
73166     120715
74005     121922
76990     126297
89707     145443
101140    161490
147196    226342
166584    253720
184082    277482
202148    301410
272903    400209
359031    516763
387005    552948
396828    565519
419634    594870
448454    630818
482802    673896
496019    690320
Name: Id, dtype: object

In [588]:
#Answers posted by Owner342
Owner342.loc[(Owner342.PostTypeId == '2'), 'Id'].head()

758     1927
787     2019
964     2513
1056    2701
1122    2800
Name: Id, dtype: object

In [571]:
byQuestionID = df.groupby('ParentId')
Nodes = dict()
for key, value in byOwnerID.groups.items():
    QA = []
    owner = byOwnerID.get_group(key)
    qs = owner.loc[(owner.PostTypeId == '1'), 'Id']
    #for each question qs, who are the people that answered the question
    for i in qs:
        qs_ans = dict()
        if i in byQuestionID.groups:
            qs_ans['ans'] = byQuestionID.get_group(i).sort_values('CreationDate')
            qs_ans['qus'] = owner.loc[(owner.Id == i),]
            QA.append(qs_ans) 
    Nodes[key] = QA

In [572]:
Time_Score = dict()
for key, value in Nodes.items():
    times = []
    if value:
        node = dict()
        for i in value:
            arr = dict()
            timeline = []
            scores = []
            if 'qus' in i:
                qusTG = i['qus']['Tags'].tolist()
                qusBD = i['qus']['Body'].tolist()
                qusTT = i['qus']['Title'].tolist()
                qusT = i['qus']['CreationDate'].tolist()
                qusS = i['qus']['Score'].tolist()
                timeline.extend(qusT)
                scores.extend(qusS)
                if 'ans' in i:
                    ansT = i['ans']['CreationDate'].tolist()
                    ansS = i['ans']['Score'].tolist()
                    timeline.extend(ansT)
                    scores.extend(ansS)
            arr['timeline'] = timeline
            arr['score'] = scores
            arr['tag'] = qusTG
            arr['body'] = qusBD
            arr['title'] = qusTT
            arr['date'] = qusT
            times.append(arr)
    Time_Score[key] = times  

In [593]:
# len(Time_Score): 40553 users
#questions for owner 9611
print (Time_Score['9611'])

[{'body': ["<p>I'm using the VBO extension for storing Vertex, normal and color buffers (glBindBufferARB)\nFor some reason when changing buffers or doing some operation the application crashes with an access violation. When attaching The debugger I see that the crash is in some thread that is not my main thread which performs the opengl call with the execution in some dll which is related to the nvidia graphics driver.</p>\n\n<p>What probably happened is that I gave some buffer call a bad buffer or with a wrong size. So my question is, how do I debug this situation? The crash seem to happen some time after the actual call and in a different thread.</p>\n"], 'title': ['Debugging VBO Vertex buffers crashes'], 'timeline': [Timestamp('2008-09-15 19:21:17.290000'), Timestamp('2008-09-15 19:35:41.217000'), Timestamp('2008-09-15 19:37:13.147000'), Timestamp('2008-09-16 16:29:36.133000')], 'tag': ['<debugging><opengl><vbo>'], 'score': ['2', '0', '-1', '3'], 'date': [Timestamp('2008-09-15 19:21

In [574]:
#time passed since first answer
fa = []
for key, value in Time_Score.items():
    if value:
        for i in value:
            d = dict()
            d['q_owner'] = key
            d['q_tag'] = i['tag'][0]
            d['q_body'] = i['body'][0]
            d['q_body_len'] = len(list(i['body'][0]))
            d['q_title'] = i['title'][0]
            d['q_date'] = i['date'][0]
            d['q_score'] = i['score'][0]
            d['fa_score'] = i['score'][1]
            d['fa_time'] = i['timeline'][1]-i['timeline'][0]
            d['fa_time'] = d['fa_time'].total_seconds()
            d['s5_time'] = None
            d['s10_time'] = None
            d['s20_time'] = None
            d['s50_time'] = None
            find5 = False
            find10 = False
            find20 = False
            find50 = False
            j=1
            while j<len(i['score']):
                if (int(i['score'][j]) >=5) and find5==False:
                    d['s5_time'] = i['timeline'][j]-i['timeline'][0]
                    d['s5_time'] = d['s5_time'].total_seconds()
                    find5 = True
                elif (int(i['score'][j]) >=10) and find10==False:
                    d['s10_time'] = i['timeline'][j]-i['timeline'][0]
                    d['s10_time'] = d['s10_time'].total_seconds()
                    find10 = True
                elif (int(i['score'][j]) >=20) and find20==False:
                    d['s20_time'] = i['timeline'][j]-i['timeline'][0]
                    d['s20_time'] = d['s20_time'].total_seconds()
                    find20 = True
                elif (int(i['score'][j]) >=50) and find50==False:
                    d['s50_time'] = i['timeline'][j]-i['timeline'][0]
                    d['s50_time'] = d['s50_time'].total_seconds()
                    find50 = True
                else:
                    j+=1
            fa.append(d)

In [575]:
# import nltk
# from bs4 import BeautifulSoup as bs
pd_fa = pd.DataFrame(fa)
pd_fa.q_date = pd.to_datetime(pd_fa.q_date)
# pd_fa.fa_time = pd.to_timedelta(pd_fa.fa_time, unit = 's').dt.seconds
# pd_fa.s5_time = pd.to_timedelta(pd_fa.s5_time, unit = 's').dt.seconds
# pd_fa.s10_time = pd.to_timedelta(pd_fa.s10_time, unit = 's').dt.seconds
# pd_fa.s20_time = pd.to_timedelta(pd_fa.s20_time, unit = 's').dt.seconds
# pd_fa.s50_time = pd.to_timedelta(pd_fa.s50_time, unit = 's').dt.seconds
pd_fa.q_score = pd.to_numeric(pd_fa.q_score)
pd_fa.q_body_len = pd.to_numeric(pd_fa.q_body_len)

In [592]:
pd_fa #18670 posts

,fa_score,fa_time,q_body,q_body_len,q_date,q_owner,q_score,q_tag,q_title,s10_time,...,c#,.net,java,asp.net,c++,javascript,sql-server,python,php,sql
0,0,236.466,<p>I am thinking along the lines of replicatin...,367,2008-09-19 05:09:25.977,essexboyracer,3,<php>,Convert/extract phpinfo() into php.ini,NaN,...,0,0,0,0,0,0,0,0,1,0
1,2,628.166,<p>Has anyone any reources for learning how to...,402,2008-09-24 06:42:46.107,essexboyracer,5,<php><mysql><svg><vector-graphics><php-gtk>,Interactive SVG - Learning Resources?,NaN,...,0,0,0,0,0,0,0,0,1,0
2,1,92052.243,<p>I'm trying to decode a WBXML encoded SyncML...,632,2008-10-22 15:25:00.777,11542,3,<python><s60><syncml><wbxml>,Decoding a WBXML SyncML message from an S60 de...,NaN,...,0,0,0,0,0,0,0,1,0,0
3,30,802.577,<p>I want to get the headers only from a curl ...,401,2008-11-13 14:00:43.840,11542,72,<curl>,Curl post data and headers only,802.577,...,0,0,0,0,0,0,0,0,0,0
4,7,806.883,<p>I'm just starting to try out phpunit on som...,651,2008-11-14 10:37:13.877,11542,3,<php><unit-testing><phpunit>,PHPUnit filenaming conventions,NaN,...,0,0,0,0,0,0,0,0,1,0
5,0,1545684.384,<p>How can UTF-8 strings (i.e. 8-bit string) b...,975,2008-09-22 23:55:02.883,11543,2,<php><html><xml><unicode><utf-8>,Convert a UTF-8 string to/from 7-bit XML in PHP,NaN,...,0,0,0,0,0,0,0,0,1,0
6,0,508.124,<p>JavaScript allows functions to be treated a...,749,2008-09-23 22:30:54.573,11543,33,<javascript><function><properties>,"How to convert an ""object"" into a function in ...",964.034,...,0,0,0,0,0,1,0,0,0,0
7,1,557.514,<p>What's the fastest way to count the number ...,381,2008-09-24 08:56:21.263,11543,1111,<javascript><performance><properties><count><key>,How to efficiently count the number of keys/pr...,901.010,...,0,0,0,0,0,1,0,0,0,0
8,2,2730.717,"<p>In <code>Eclipse PDT</code>, <code>Ctrl-Shi...",984,2008-09-25 10:27:52.750,11543,10,<php><eclipse><comments><eclipse-pdt><reformat>,How to reformat multi-line comments in Eclipse...,NaN,...,0,0,0,0,0,0,0,0,1,0
9,0,228.570,<p>How do you do case conversion in XSL?</p>\n...,184,2009-02-25 14:43:02.327,11543,100,<string><xpath><xslt><case-conversion>,How can I convert a string to upper- or lower-...,401.263,...,0,0,0,0,0,0,0,0,0,0


In [577]:
pd_fa.describe()

,fa_time,q_body_len,q_score,s10_time,s20_time,s50_time,s5_time
count,1.088310e+05,108831.000000,108831.000000,3.297900e+04,1.856400e+04,8.484000e+03,5.537900e+04
mean,6.824459e+04,826.968548,23.547978,1.492718e+05,1.810815e+05,2.134462e+05,1.250337e+05
std,5.648813e+05,897.134121,141.887646,1.045164e+06,1.183872e+06,1.318806e+06,9.119214e+05
min,-1.223458e+07,18.000000,-7.000000,-1.223458e+07,-1.223458e+07,-1.223458e+07,-1.223458e+07
25%,2.629085e+02,342.000000,1.000000,2.661535e+02,2.672228e+02,2.531335e+02,2.760905e+02
50%,5.804500e+02,598.000000,3.000000,6.338600e+02,6.457880e+02,6.311095e+02,6.461700e+02
75%,2.085712e+03,1018.000000,10.000000,2.535225e+03,2.713106e+03,2.802904e+03,2.539932e+03
max,1.913022e+07,34183.000000,9484.000000,1.861181e+07,1.952016e+07,1.952016e+07,1.748575e+07


In [578]:
tag = dict()
for i in pd_fa.q_tag:
    li = i.split('><')
    for j in li:
        j = j.replace('<', '')
        j = j.replace('>', '')
        if j not in tag:
            tag[j] = 1
        else:
            tag[j] += 1

In [579]:
sortedTags = sorted(tag.items(), key = operator.itemgetter(1), reverse = True)

In [580]:
morethan100 = {k:v for k, v in tag.items() if v>=100}

In [581]:
len(morethan100)

466

In [582]:
morethan100

{'.htaccess': 152,
 '.net': 10512,
 '.net-2.0': 459,
 '.net-3.5': 570,
 '3d': 139,
 '64bit': 229,
 'actionscript': 298,
 'actionscript-3': 656,
 'active-directory': 210,
 'activerecord': 175,
 'activex': 131,
 'ado.net': 318,
 'agile': 166,
 'air': 161,
 'ajax': 1185,
 'algorithm': 1108,
 'android': 152,
 'animation': 142,
 'ant': 221,
 'apache': 668,
 'api': 329,
 'applet': 125,
 'architecture': 471,
 'arrays': 782,
 'asp-classic': 317,
 'asp.net': 7212,
 'asp.net-2.0': 159,
 'asp.net-ajax': 280,
 'asp.net-mvc': 1959,
 'assemblies': 194,
 'assembly': 168,
 'asynchronous': 162,
 'attributes': 134,
 'audio': 255,
 'authentication': 416,
 'autocomplete': 109,
 'automated-tests': 104,
 'automation': 207,
 'backup': 160,
 'bash': 458,
 'batch-file': 284,
 'binary': 121,
 'binding': 146,
 'boost': 211,
 'browser': 400,
 'build': 142,
 'build-automation': 186,
 'build-process': 261,
 'c': 2207,
 'c#': 15092,
 'c#-3.0': 149,
 'c++': 5771,
 'caching': 441,
 'calendar': 101,
 'casting': 178,
 '

In [583]:
sortedTags[0:10]

[('c#', 15092),
 ('.net', 10512),
 ('java', 7855),
 ('asp.net', 7212),
 ('c++', 5771),
 ('javascript', 5307),
 ('php', 4458),
 ('python', 4005),
 ('sql', 3857),
 ('sql-server', 3658)]

In [584]:
pd_fa['c#'] = 0
pd_fa['.net'] = 0
pd_fa['java'] = 0
pd_fa['asp.net'] = 0
pd_fa['c++'] = 0
pd_fa['javascript'] = 0
pd_fa['sql-server'] = 0
pd_fa['python'] = 0
pd_fa['php'] = 0
pd_fa['sql'] = 0

In [585]:
def findtag(c, s):
    if re.search(c, s) != None:
        return 1
    else:
        return 0
pd_fa['c#'] = pd_fa.apply(lambda row: findtag('<c#>', row['q_tag']), axis = 1)
pd_fa['c++'] = pd_fa.apply(lambda row: findtag('<c\+\+>', row['q_tag']), axis = 1)
pd_fa['.net'] = pd_fa.apply(lambda row: findtag('<.net>', row['q_tag']), axis = 1)
pd_fa['java'] = pd_fa.apply(lambda row: findtag('<java>', row['q_tag']), axis = 1)
pd_fa['asp.net'] = pd_fa.apply(lambda row: findtag('<asp.net>', row['q_tag']), axis = 1)
pd_fa['javascript'] = pd_fa.apply(lambda row: findtag('<javascript>', row['q_tag']), axis = 1)
pd_fa['sql-server'] = pd_fa.apply(lambda row: findtag('<sql-server>', row['q_tag']), axis = 1)
pd_fa['python'] = pd_fa.apply(lambda row: findtag('<python>', row['q_tag']), axis = 1)
pd_fa['php'] = pd_fa.apply(lambda row: findtag('<php>', row['q_tag']), axis = 1)
pd_fa['sql'] = pd_fa.apply(lambda row: findtag('<sql>', row['q_tag']), axis = 1)

In [586]:
pd_fa

,fa_score,fa_time,q_body,q_body_len,q_date,q_owner,q_score,q_tag,q_title,s10_time,...,c#,.net,java,asp.net,c++,javascript,sql-server,python,php,sql
0,0,236.466,<p>I am thinking along the lines of replicatin...,367,2008-09-19 05:09:25.977,essexboyracer,3,<php>,Convert/extract phpinfo() into php.ini,NaN,...,0,0,0,0,0,0,0,0,1,0
1,2,628.166,<p>Has anyone any reources for learning how to...,402,2008-09-24 06:42:46.107,essexboyracer,5,<php><mysql><svg><vector-graphics><php-gtk>,Interactive SVG - Learning Resources?,NaN,...,0,0,0,0,0,0,0,0,1,0
2,1,92052.243,<p>I'm trying to decode a WBXML encoded SyncML...,632,2008-10-22 15:25:00.777,11542,3,<python><s60><syncml><wbxml>,Decoding a WBXML SyncML message from an S60 de...,NaN,...,0,0,0,0,0,0,0,1,0,0
3,30,802.577,<p>I want to get the headers only from a curl ...,401,2008-11-13 14:00:43.840,11542,72,<curl>,Curl post data and headers only,802.577,...,0,0,0,0,0,0,0,0,0,0
4,7,806.883,<p>I'm just starting to try out phpunit on som...,651,2008-11-14 10:37:13.877,11542,3,<php><unit-testing><phpunit>,PHPUnit filenaming conventions,NaN,...,0,0,0,0,0,0,0,0,1,0
5,0,1545684.384,<p>How can UTF-8 strings (i.e. 8-bit string) b...,975,2008-09-22 23:55:02.883,11543,2,<php><html><xml><unicode><utf-8>,Convert a UTF-8 string to/from 7-bit XML in PHP,NaN,...,0,0,0,0,0,0,0,0,1,0
6,0,508.124,<p>JavaScript allows functions to be treated a...,749,2008-09-23 22:30:54.573,11543,33,<javascript><function><properties>,"How to convert an ""object"" into a function in ...",964.034,...,0,0,0,0,0,1,0,0,0,0
7,1,557.514,<p>What's the fastest way to count the number ...,381,2008-09-24 08:56:21.263,11543,1111,<javascript><performance><properties><count><key>,How to efficiently count the number of keys/pr...,901.010,...,0,0,0,0,0,1,0,0,0,0
8,2,2730.717,"<p>In <code>Eclipse PDT</code>, <code>Ctrl-Shi...",984,2008-09-25 10:27:52.750,11543,10,<php><eclipse><comments><eclipse-pdt><reformat>,How to reformat multi-line comments in Eclipse...,NaN,...,0,0,0,0,0,0,0,0,1,0
9,0,228.570,<p>How do you do case conversion in XSL?</p>\n...,184,2009-02-25 14:43:02.327,11543,100,<string><xpath><xslt><case-conversion>,How can I convert a string to upper- or lower-...,401.263,...,0,0,0,0,0,0,0,0,0,0


In [587]:
pd_fa.to_csv('viz/time_score.csv', encoding='utf-8')

In [599]:
pd_fa.loc[pd_fa['.net'] == 1, ].describe()

,fa_time,q_body_len,q_score,s10_time,s20_time,s50_time,s5_time,c#,.net,java,asp.net,c++,javascript,sql-server,python,php,sql
count,1.051300e+04,10513.000000,10513.000000,3.181000e+03,1.784000e+03,7.860000e+02,5.407000e+03,10513.000000,10513.0,10513.000000,10513.000000,10513.000000,10513.000000,10513.000000,10513.000000,10513.000000,10513.000000
mean,6.372157e+04,831.339294,19.069152,1.813348e+05,2.307420e+05,3.197335e+05,1.467764e+05,0.423190,1.0,0.011700,0.071626,0.009227,0.006468,0.018644,0.001712,0.003044,0.014744
std,5.508713e+05,854.182427,92.212588,1.184385e+06,1.349262e+06,1.655248e+06,1.059819e+06,0.494089,0.0,0.107536,0.257879,0.095616,0.080168,0.135269,0.041345,0.055090,0.120531
min,-3.900118e+06,18.000000,-3.000000,1.171300e+01,1.171300e+01,1.477400e+01,-3.897671e+06,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.314270e+02,345.000000,1.000000,2.394300e+02,2.538350e+02,2.463973e+02,2.458730e+02,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,4.783430e+02,603.000000,4.000000,5.328800e+02,5.749635e+02,6.362785e+02,5.342530e+02,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.521463e+03,1038.000000,11.000000,1.841000e+03,2.148882e+03,2.942757e+03,1.856212e+03,1.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.333501e+07,27787.000000,5009.000000,1.849535e+07,1.935003e+07,1.935003e+07,1.705095e+07,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [597]:
pd_fa.loc[pd_fa['asp.net'] == 1, ].describe()

,fa_time,q_body_len,q_score,s10_time,s20_time,s50_time,s5_time,c#,.net,java,asp.net,c++,javascript,sql-server,python,php,sql
count,7.212000e+03,7212.000000,7212.000000,1.194000e+03,5.500000e+02,2.000000e+02,2.472000e+03,7212.000000,7212.000000,7212.000000,7212.0,7212.000000,7212.000000,7212.000000,7212.000000,7212.000000,7212.000000
mean,5.173580e+04,888.214226,7.811009,2.245837e+05,3.856031e+05,3.989130e+05,1.838260e+05,0.229340,0.104409,0.001109,1.0,0.000139,0.065308,0.020105,0.000416,0.004992,0.014836
std,4.705463e+05,940.610601,38.690716,1.294606e+06,1.845047e+06,1.909906e+06,1.103164e+06,0.420437,0.305812,0.033289,0.0,0.011775,0.247085,0.140371,0.020393,0.070480,0.120906
min,-3.900118e+06,20.000000,-5.000000,2.369700e+01,2.369700e+01,3.982600e+01,-3.897671e+06,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.714835e+02,374.000000,1.000000,2.747425e+02,3.095705e+02,2.850685e+02,2.860275e+02,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,5.804165e+02,650.000000,2.000000,7.282000e+02,8.428970e+02,7.283105e+02,7.086415e+02,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.836970e+03,1092.000000,5.000000,3.159433e+03,4.086067e+03,3.055396e+03,2.619608e+03,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.374776e+07,18642.000000,1736.000000,1.701374e+07,1.952016e+07,1.952016e+07,1.701374e+07,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [598]:
pd_fa.loc[pd_fa['python'] == 1, ].describe()

,fa_time,q_body_len,q_score,s10_time,s20_time,s50_time,s5_time,c#,.net,java,asp.net,c++,javascript,sql-server,python,php,sql
count,4.005000e+03,4005.000000,4005.000000,1.988000e+03,1.260000e+03,6.630000e+02,2.853000e+03,4005.000000,4005.000000,4005.000000,4005.000000,4005.000000,4005.000000,4005.000000,4005.0,4005.000000,4005.000000
mean,2.474758e+04,822.070412,58.022722,8.891482e+04,1.190259e+05,1.309070e+05,6.669666e+04,0.008489,0.004494,0.015980,0.000749,0.011985,0.005993,0.001498,1.0,0.011985,0.007740
std,3.744471e+05,763.837778,259.716098,9.308768e+05,1.041535e+06,1.117578e+06,7.380356e+05,0.091757,0.066898,0.125414,0.027362,0.108832,0.077189,0.038681,0.0,0.108832,0.087649
min,-1.223458e+07,22.000000,-4.000000,-1.223458e+07,-1.223458e+07,-1.223458e+07,-1.223458e+07,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.000000
25%,2.909430e+02,342.000000,3.000000,2.977668e+02,2.799045e+02,2.634315e+02,3.102370e+02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.000000
50%,6.409770e+02,598.000000,7.000000,7.307400e+02,7.134700e+02,6.516270e+02,7.661540e+02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.000000
75%,2.092880e+03,1030.000000,24.000000,2.929093e+03,3.213816e+03,2.858694e+03,2.826564e+03,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.000000
max,1.056742e+07,8615.000000,7737.000000,1.701160e+07,1.620498e+07,1.213580e+07,1.620498e+07,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000
